# 三张表： user.csv, click_log.csv, ad.csv
- age	gender
- creative_id	ad_id	product_id	product_category	advertiser_id	industry

素材id、广告id、产品id、产品类目id、广告主id、广告主行业id 

- time	user_id	creative_id	click_times

户当天点击该广告的次数

In [ ]:
import pandas as pd
from pandas import DataFrame
from gensim.models import word2vec
import gensim
from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',60)


user_df = pd.read_csv('user.csv')
click_log_df = pd.read_csv('click_log.csv')
ad_df = pd.read_csv('ad.csv')

### merge到click_log表

df_train = user_df.merge(click_log_df,on='user_id',how='left')
df_train = df_train.merge(ad_df,on='creative_id',how='left')

print(len(user_df))
print(len(click_log_df))
print(len(ad_df))

print(user_df.columns.tolist())
print(click_log_df.columns.tolist())
print(ad_df.columns.tolist())

print(user_df.dtypes)
print(click_log_df.dtypes)
print(ad_df.dtypes) # industry: 326 个类别, 有 \\N   product_id: 33273个类别,有 \\N 

len(df_train)

- print(len(set(ad_df['creative_id']))) # 2481135
- print(len(set(ad_df['ad_id']))) # 2264190
- print(len(set(ad_df['product_id']))) # 33273
- print(len(set(ad_df['product_category']))) # 18
- print(len(set(ad_df['advertiser_id']))) # 52090
- print(len(set(ad_df['industry']))) # 326

In [ ]:
click_log_df_test = pd.read_csv('../test/click_log.csv')
ad_df_test = pd.read_csv('../test/ad.csv')
df_test = click_log_df_test.merge(ad_df_test,on='creative_id',how='left')

In [ ]:
df_train['product_id'] = df_train['product_id'].replace('\\N',-999)
df_train['industry'] = df_train['industry'].replace('\\N',-999)


In [ ]:
df_train[df_train['user_id']==2]['age'].values[0]

In [ ]:
creative_id_df = DataFrame()
id_list = user_df['user_id'].tolist()
age_list = user_df['age'].tolist()
gender_list = user_df['gender'].tolist()
creative_id_list = []

class build_creation_id(object):
    def __init__(self):
        pass
    def __iter__(self):
        for ids in tqdm(user_df['user_id']):
            #df_train[df_train['user_id']==ids].sort_values(by="time" , ascending=True)
            #creative_id_list.append(df_train[df_train['user_id']==ids].sort_values(by="time" , ascending=True)['creative_id'].tolist())
            yield df_train[df_train['user_id']==ids].sort_values(by="time" , ascending=True)['creative_id'].apply(str).tolist()
model = gensim.models.Word2Vec(build_creation_id())

In [ ]:
creative_id_df = DataFrame()
id_list = user_df['user_id'].tolist()
age_list = user_df['age'].tolist()
gender_list = user_df['gender'].tolist()
creative_id_list = []

for ids in tqdm(user_df['user_id']):
        #df_train[df_train['user_id']==ids].sort_values(by="time" , ascending=True)
        #creative_id_list.append(df_train[df_train['user_id']==ids].sort_values(by="time" , ascending=True)['creative_id'].tolist())
    creative_id_list.append(df_train[df_train['user_id']==ids].sort_values(by="time" , ascending=True)['creative_id'].apply(str).tolist())


In [ ]:
creative_id_df = DataFrame({'id':id_list,'age':age_list,'gender':gender_list,'sentence':creative_id_list})
creative_id_df.to_csv('creative_id_df.csv')

In [ ]:
print('素材id、广告id、产品id、产品类目id、广告主id、广告主行业id ')
df_test.head()

In [ ]:
df_train['label'] = df_train['age']+10*df_train['gender']

In [ ]:
for ids in df_train['user_id']:
    df_train[df_train['id']==ids][creative_id]

In [ ]:
set(df_train['gender'])

In [ ]:
set(df_train['age'])

len(df_train[df_train['product_id']=='\\N']) #1259, 3112

len(df_train[df_train['industry']=='\\N']) # 127,6874

In [ ]:
X = df_train.drop(['user_id','age','gender'], 1)
Y = df_train['label']

In [ ]:
len(Y)

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import numpy as np
train_data = lgb.Dataset(X, label=Y)
X_train,X_val,y_train,y_val = train_test_split(X,Y,test_size=0.2)
xgtrain = lgb.Dataset(X_train, y_train)
xgvalid = lgb.Dataset(X_val, y_val)



w = np.random.rand(30082771, )
# train_data = lgb.Dataset(data, label=label, feature_name=['c1', 'c2', 'c3'], 
#                    categorical_feature=['c3'],weight=w)


train_data.set_weight(w)

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary', #xentlambda
    'metric': 'auc',
    'silent':0,
    'learning_rate': 0.05,
    'is_unbalance': 'true',  #当训练数据是不平衡的，正负样本相差悬殊的时候，可以将这个属性设为true,此时会自动给少的样本赋予更高的权重
    'num_leaves': 64,  # 一般设为少于2^(max_depth)
    'max_depth': -1,  #最大的树深，设为-1时表示不限制树的深度
    'min_child_samples': 15,  # 每个叶子结点最少包含的样本数量，用于正则化，避免过拟合
    'max_bin': 200,  # 设置连续特征或大量类型的离散特征的bins的数量
    'subsample': 0.8,  # Subsample ratio of the training instance.
    'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
    'colsample_bytree': 0.5,  # Subsample ratio of columns when constructing each tree.
    'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
    #'scale_pos_weight':100,
    'subsample_for_bin': 200000,  # Number of samples for constructing bin
    'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
    'reg_alpha': 2.99,  # L1 regularization term on weights
    'reg_lambda': 1.9,  # L2 regularization term on weights
    'nthread': 10,
    'verbose': 0,
}

In [ ]:
def feval_spec(preds, train_data):
    from sklearn.metrics import roc_curve
    fpr, tpr, threshold = roc_curve(train_data.get_label(), preds)
    tpr0001 = tpr[fpr <= 0.0005].max()
    tpr001 = tpr[fpr <= 0.001].max()
    tpr005 = tpr[fpr <= 0.005].max()
    #tpr01 = tpr[fpr.values <= 0.01].max()
    tprcal = 0.4 * tpr0001 + 0.3 * tpr001 + 0.3 * tpr005
    return 'spec_cal',tprcal,True



num_round = 10
bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])

In [ ]:
num_round = 10
lgb.cv(param, train_data, num_round, nfold=5)

bst = lgb.train(param, train_data, num_round, valid_sets=valid_sets, 
      early_stopping_rounds=10)
bst.save_model('model.txt', num_iteration=bst.best_iteration)


data = np.random.rand(7, 10)
ypred = bst.predict(data)

ypred = bst.predict(data, num_iteration=bst.best_iteration)

# 参考
- tfidf

In [2]:
import pandas as pd
creation = pd.read_csv('creative_id_df.csv')
creation.head()

sentences=[]
for item in creation['sentence']:
    #sent = ''.join(item[1:-1].split(','))
    sent = list(item[1:-1].replace("'",'').split(','))
    sentences.append(sent)

In [ ]:
len(sentences)

In [3]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import multiprocessing
from gensim.models import Word2Vec
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
w2v_model = Word2Vec(min_count=1,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

w2v_model.corpus_count

t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=5, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 22:55:48: collecting all words and their counts
INFO - 22:55:48: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 22:55:48: PROGRESS: at sentence #10000, processed 335578 words, keeping 153379 word types
INFO - 22:55:48: PROGRESS: at sentence #20000, processed 673775 words, keeping 259099 word types
INFO - 22:55:48: PROGRESS: at sentence #30000, processed 1012932 words, keeping 347138 word types
INFO - 22:55:48: PROGRESS: at sentence #40000, processed 1344506 words, keeping 423601 word types
INFO - 22:55:48: PROGRESS: at sentence #50000, processed 1683247 words, keeping 495547 word types
INFO - 22:55:48: PROGRESS: at sentence #60000, processed 2018755 words, keeping 560179 word types
INFO - 22:55:48: PROGRESS: at sentence #70000, processed 2356409 words, keeping 621113 word types
INFO - 22:55:49: PROGRESS: at sentence #80000, processed 2694811 words, keeping 677894 word types
INFO - 22:55:49: PROGRESS: at sentence #90000, processed 3029095 words, keeping 

INFO - 22:55:57: PROGRESS: at sentence #820000, processed 27402382 words, keeping 2517129 word types
INFO - 22:55:57: PROGRESS: at sentence #830000, processed 27732476 words, keeping 2531438 word types
INFO - 22:55:57: PROGRESS: at sentence #840000, processed 28080276 words, keeping 2546875 word types
INFO - 22:55:57: PROGRESS: at sentence #850000, processed 28417412 words, keeping 2561301 word types
INFO - 22:55:57: PROGRESS: at sentence #860000, processed 28751550 words, keeping 2575644 word types
INFO - 22:55:57: PROGRESS: at sentence #870000, processed 29085965 words, keeping 2589918 word types
INFO - 22:55:57: PROGRESS: at sentence #880000, processed 29420540 words, keeping 2603695 word types
INFO - 22:55:57: PROGRESS: at sentence #890000, processed 29751798 words, keeping 2617206 word types
INFO - 22:55:57: collected 2630643 word types from a corpus of 30082771 raw words and 900000 sentences
INFO - 22:55:57: Loading a fresh vocabulary
INFO - 22:56:05: effective_min_count=1 retain

Time to build vocab: 1.03 mins


INFO - 22:56:51: EPOCH 1 - PROGRESS: at 0.72% examples, 201176 words/s, in_qsize 9, out_qsize 0
INFO - 22:56:52: EPOCH 1 - PROGRESS: at 1.72% examples, 236158 words/s, in_qsize 9, out_qsize 0
INFO - 22:56:53: EPOCH 1 - PROGRESS: at 2.72% examples, 250639 words/s, in_qsize 10, out_qsize 0
INFO - 22:56:54: EPOCH 1 - PROGRESS: at 3.76% examples, 258530 words/s, in_qsize 10, out_qsize 0
INFO - 22:56:55: EPOCH 1 - PROGRESS: at 4.71% examples, 259007 words/s, in_qsize 10, out_qsize 0
INFO - 22:56:56: EPOCH 1 - PROGRESS: at 5.63% examples, 258470 words/s, in_qsize 9, out_qsize 0
INFO - 22:56:57: EPOCH 1 - PROGRESS: at 6.65% examples, 260424 words/s, in_qsize 10, out_qsize 0
INFO - 22:56:58: EPOCH 1 - PROGRESS: at 7.66% examples, 261870 words/s, in_qsize 10, out_qsize 0
INFO - 22:56:59: EPOCH 1 - PROGRESS: at 8.65% examples, 262597 words/s, in_qsize 10, out_qsize 0
INFO - 22:57:00: EPOCH 1 - PROGRESS: at 9.65% examples, 263851 words/s, in_qsize 10, out_qsize 1
INFO - 22:57:01: EPOCH 1 - PROGRE

INFO - 22:58:20: EPOCH 1 - PROGRESS: at 91.08% examples, 279553 words/s, in_qsize 8, out_qsize 1
INFO - 22:58:21: EPOCH 1 - PROGRESS: at 92.23% examples, 279899 words/s, in_qsize 9, out_qsize 0
INFO - 22:58:22: EPOCH 1 - PROGRESS: at 93.26% examples, 279717 words/s, in_qsize 9, out_qsize 0
INFO - 22:58:23: EPOCH 1 - PROGRESS: at 94.33% examples, 279957 words/s, in_qsize 10, out_qsize 0
INFO - 22:58:24: EPOCH 1 - PROGRESS: at 95.38% examples, 279866 words/s, in_qsize 8, out_qsize 1
INFO - 22:58:25: EPOCH 1 - PROGRESS: at 96.54% examples, 279947 words/s, in_qsize 8, out_qsize 1
INFO - 22:58:26: EPOCH 1 - PROGRESS: at 97.69% examples, 280037 words/s, in_qsize 9, out_qsize 1
INFO - 22:58:28: EPOCH 1 - PROGRESS: at 98.86% examples, 280127 words/s, in_qsize 8, out_qsize 1
INFO - 22:58:28: worker thread finished; awaiting finish of 4 more threads
INFO - 22:58:28: worker thread finished; awaiting finish of 3 more threads
INFO - 22:58:28: worker thread finished; awaiting finish of 2 more thread

INFO - 22:59:49: EPOCH 2 - PROGRESS: at 84.04% examples, 290694 words/s, in_qsize 9, out_qsize 0
INFO - 22:59:50: EPOCH 2 - PROGRESS: at 85.20% examples, 290693 words/s, in_qsize 8, out_qsize 1
INFO - 22:59:51: EPOCH 2 - PROGRESS: at 86.35% examples, 290720 words/s, in_qsize 10, out_qsize 0
INFO - 22:59:52: EPOCH 2 - PROGRESS: at 87.52% examples, 290740 words/s, in_qsize 10, out_qsize 0
INFO - 22:59:53: EPOCH 2 - PROGRESS: at 88.70% examples, 290723 words/s, in_qsize 8, out_qsize 1
INFO - 22:59:54: EPOCH 2 - PROGRESS: at 89.85% examples, 290745 words/s, in_qsize 8, out_qsize 1
INFO - 22:59:55: EPOCH 2 - PROGRESS: at 91.02% examples, 290763 words/s, in_qsize 8, out_qsize 1
INFO - 22:59:56: EPOCH 2 - PROGRESS: at 92.19% examples, 290751 words/s, in_qsize 10, out_qsize 1
INFO - 22:59:58: EPOCH 2 - PROGRESS: at 93.33% examples, 290782 words/s, in_qsize 10, out_qsize 0
INFO - 22:59:59: EPOCH 2 - PROGRESS: at 94.48% examples, 290799 words/s, in_qsize 8, out_qsize 1
INFO - 23:00:00: EPOCH 2 -

INFO - 23:01:16: EPOCH 3 - PROGRESS: at 76.56% examples, 293909 words/s, in_qsize 10, out_qsize 0
INFO - 23:01:17: EPOCH 3 - PROGRESS: at 77.71% examples, 293940 words/s, in_qsize 10, out_qsize 0
INFO - 23:01:18: EPOCH 3 - PROGRESS: at 78.87% examples, 293854 words/s, in_qsize 9, out_qsize 0
INFO - 23:01:19: EPOCH 3 - PROGRESS: at 80.03% examples, 293867 words/s, in_qsize 10, out_qsize 1
INFO - 23:01:20: EPOCH 3 - PROGRESS: at 81.18% examples, 293894 words/s, in_qsize 9, out_qsize 0
INFO - 23:01:22: EPOCH 3 - PROGRESS: at 82.34% examples, 293911 words/s, in_qsize 10, out_qsize 0
INFO - 23:01:23: EPOCH 3 - PROGRESS: at 83.51% examples, 293956 words/s, in_qsize 10, out_qsize 0
INFO - 23:01:24: EPOCH 3 - PROGRESS: at 84.67% examples, 294001 words/s, in_qsize 9, out_qsize 0
INFO - 23:01:25: EPOCH 3 - PROGRESS: at 85.82% examples, 294015 words/s, in_qsize 10, out_qsize 0
INFO - 23:01:26: EPOCH 3 - PROGRESS: at 87.00% examples, 294051 words/s, in_qsize 9, out_qsize 0
INFO - 23:01:27: EPOCH 3

INFO - 23:02:40: EPOCH 4 - PROGRESS: at 66.08% examples, 293972 words/s, in_qsize 9, out_qsize 0
INFO - 23:02:41: EPOCH 4 - PROGRESS: at 67.25% examples, 294072 words/s, in_qsize 10, out_qsize 0
INFO - 23:02:42: EPOCH 4 - PROGRESS: at 68.41% examples, 294183 words/s, in_qsize 9, out_qsize 0
INFO - 23:02:43: EPOCH 4 - PROGRESS: at 69.57% examples, 294306 words/s, in_qsize 10, out_qsize 0
INFO - 23:02:45: EPOCH 4 - PROGRESS: at 70.73% examples, 294393 words/s, in_qsize 9, out_qsize 0
INFO - 23:02:46: EPOCH 4 - PROGRESS: at 71.89% examples, 294488 words/s, in_qsize 9, out_qsize 0
INFO - 23:02:47: EPOCH 4 - PROGRESS: at 73.05% examples, 294566 words/s, in_qsize 9, out_qsize 0
INFO - 23:02:48: EPOCH 4 - PROGRESS: at 74.17% examples, 294792 words/s, in_qsize 9, out_qsize 0
INFO - 23:02:49: EPOCH 4 - PROGRESS: at 75.26% examples, 294710 words/s, in_qsize 9, out_qsize 0
INFO - 23:02:50: EPOCH 4 - PROGRESS: at 76.33% examples, 294689 words/s, in_qsize 8, out_qsize 1
INFO - 23:02:51: EPOCH 4 - P

INFO - 23:04:06: EPOCH 5 - PROGRESS: at 58.76% examples, 298104 words/s, in_qsize 10, out_qsize 1
INFO - 23:04:08: EPOCH 5 - PROGRESS: at 59.92% examples, 298111 words/s, in_qsize 10, out_qsize 1
INFO - 23:04:09: EPOCH 5 - PROGRESS: at 61.07% examples, 298124 words/s, in_qsize 10, out_qsize 0
INFO - 23:04:10: EPOCH 5 - PROGRESS: at 62.25% examples, 298117 words/s, in_qsize 10, out_qsize 0
INFO - 23:04:11: EPOCH 5 - PROGRESS: at 63.43% examples, 298101 words/s, in_qsize 10, out_qsize 0
INFO - 23:04:12: EPOCH 5 - PROGRESS: at 64.59% examples, 298082 words/s, in_qsize 9, out_qsize 0
INFO - 23:04:13: EPOCH 5 - PROGRESS: at 65.76% examples, 298078 words/s, in_qsize 9, out_qsize 0
INFO - 23:04:14: EPOCH 5 - PROGRESS: at 66.91% examples, 298085 words/s, in_qsize 8, out_qsize 1
INFO - 23:04:15: EPOCH 5 - PROGRESS: at 68.08% examples, 298076 words/s, in_qsize 9, out_qsize 0
INFO - 23:04:16: EPOCH 5 - PROGRESS: at 69.24% examples, 298017 words/s, in_qsize 8, out_qsize 1
INFO - 23:04:17: EPOCH 5 

Time to train the model: 7.92 mins


In [4]:
w2v_model.init_sims(replace=True)

INFO - 23:04:45: precomputing L2-norms of word weight vectors


In [5]:
w2v_model.wv.most_similar(positive=["1223"])

[(' 777148', 0.9957123398780823),
 (' 2300520', 0.9950451850891113),
 (' 4374155', 0.9949368238449097),
 (' 96687', 0.9949037432670593),
 (' 237751', 0.9948924779891968),
 (' 125680', 0.9948118925094604),
 ('490972', 0.9947587847709656),
 (' 222040', 0.9947294592857361),
 (' 435810', 0.9947066903114319),
 (' 567535', 0.9947012662887573)]

In [6]:
w2v_model.wv['260644']

array([ 0.02852395,  0.12255157,  0.00364791,  0.1142091 ,  0.02547306,
        0.03763156,  0.06508412,  0.01677829,  0.05980371,  0.00129941,
       -0.05996538,  0.09917008,  0.01787447,  0.06870812, -0.00550458,
        0.03171371, -0.00551774,  0.07187467,  0.02806522, -0.0625302 ,
        0.03758309, -0.0647551 , -0.08147276, -0.00129494,  0.13440752,
       -0.07413115,  0.04718652, -0.00169556,  0.04430559,  0.00626139,
       -0.10673538,  0.05980246, -0.06952408, -0.05417531,  0.00287848,
        0.02553058, -0.00341029,  0.03896419,  0.01680904, -0.03892746,
       -0.00174816,  0.02242888, -0.04416034, -0.04266509, -0.04192096,
       -0.03992682, -0.01612154,  0.00453351, -0.00509735, -0.02759567,
        0.1687546 , -0.10769073, -0.06315976,  0.02836495, -0.04426255,
        0.08104195,  0.17512217,  0.01137067, -0.0193985 , -0.04283449,
       -0.0085313 ,  0.01186987, -0.01000066,  0.07693687,  0.04670606,
       -0.05314675,  0.06060739, -0.04227314, -0.02658184, -0.01

In [13]:
#w2v_model.save('w2v.model')
w2v_model.wv.save_word2vec_format('model.bin', binary=True)


INFO - 23:11:10: storing 2630643x300 projection weights into model.bin


In [80]:
import numpy as np

In [85]:
w2v_model = gensim.models.Word2Vec.load('w2v.model') 

## 2 构造包含所有词语的 list，以及初始化 “词语-序号”字典 和 “词向量”矩阵
vocab_list = [word for word, Vocab in w2v_model.wv.vocab.items()]# 存储 所有的 词语

word_index = {" ": 0}# 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
word_vector = {} # 初始化`[word : vector]`字典

# 初始化存储所有向量的大矩阵，留意其中多一位（首行），词向量全为 0，用于 padding补零。
# 行数 为 所有单词数+1 比如 10000+1 ； 列数为 词向量“维度”比如100。
embeddings_matrix = np.zeros((len(vocab_list) + 1, w2v_model.vector_size))

#embeddings_matrix.shape

## 3 填充 上述 的字典 和 大矩阵
for i in range(len(vocab_list)):
    # print(i)
    word = vocab_list[i]  # 每个词语
    word_index[word] = i + 1 # 词语：序号
    word_vector[word] = w2v_model.wv[word] # 词语：词向量
    embeddings_matrix[i + 1] = w2v_model.wv[word]  # 词向量矩阵

INFO - 22:29:59: loading Word2Vec object from w2v.model
INFO - 22:30:07: loading wv recursively from w2v.model.wv.* with mmap=None
INFO - 22:30:07: loading vectors from w2v.model.wv.vectors.npy with mmap=None
INFO - 22:30:33: setting ignored attribute vectors_norm to None
INFO - 22:30:33: loading vocabulary recursively from w2v.model.vocabulary.* with mmap=None
INFO - 22:30:33: loading trainables recursively from w2v.model.trainables.* with mmap=None
INFO - 22:30:33: loading syn1neg from w2v.model.trainables.syn1neg.npy with mmap=None
INFO - 22:30:59: setting ignored attribute cum_table to None
INFO - 22:30:59: loaded w2v.model


In [107]:
from keras.layers import Embedding

EMBEDDING_DIM = 300 
MAX_SEQUENCE_LENGTH = 500
# 字典长度  # 词向量 长度（100） # 重点：预训练的词向量系数# 每句话的 最大长度（必须padding）  # 是否在 训练的过程中 更新词向量
embedding_layer = Embedding(input_dim = len(embeddings_matrix), 
                            output_dim=EMBEDDING_DIM,
                            weights=[embeddings_matrix], 
                            input_length=MAX_SEQUENCE_LENGTH, 
                            trainable=False)

In [87]:
creation.head()

,Unnamed: 0,id,age,gender,sentence
0,0,1,4,1,"['821396', '209778', '877468', '1683713', '122..."
1,1,2,10,1,"['63441', '155822', '39714', '609050', '126618..."
2,2,3,7,2,"['661347', '808612', '710859', '825434', '5935..."
3,3,4,5,1,"['39588', '589886', '574787', '1892854', '1962..."
4,4,5,4,1,"['296145', '350759', '24333', '43235', '852327..."


In [92]:
y_gender = list(creation['gender'])

In [104]:
creation['gender'].value_counts()

1    602610
2    297390
Name: gender, dtype: int64

In [102]:
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [105]:
MAX_SEQUENCE_LENGTH = 500
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

act = 'relu'
re_weight=True

In [108]:
lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

merged = concatenate([x1, y1])
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

########################################
## add class weight
########################################
if re_weight:
    class_weight = {1: 2, 2: 1}
else:
    class_weight = None

InternalError: 2 root error(s) found.
  (0) Internal: Dst tensor is not initialized.
	 [[{{node _arg_embedding_1/Placeholder_0_0}}]]
	 [[embedding_1/ReadVariableOp/_7]]
  (1) Internal: Dst tensor is not initialized.
	 [[{{node _arg_embedding_1/Placeholder_0_0}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
########################################
## train the model
########################################
model = Model(inputs=[sequence_1_input, sequence_2_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])
#model.summary()
print(STAMP)

early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit([data_1_train, data_2_train], labels_train, \
        validation_data=([data_1_val, data_2_val], labels_val, weight_val), \
        epochs=200, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])

########################################
## make the submission
########################################
print('Start making the submission before fine-tuning')

preds = model.predict([test_data_1, test_data_2], batch_size=8192, verbose=1)
preds += model.predict([test_data_2, test_data_1], batch_size=8192, verbose=1)
preds /= 2

submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':preds.ravel()})
submission.to_csv('%.4f_'%(bst_val_score)+STAMP+'.csv', index=False)

model.save_word2vec_format('model.bin', binary=True)

from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format('w2v.model',binary=False,encoding='utf-8', unicode_errors='ignore')
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
"I come to China to travel",
"This is a car polupar in China",
"I love tea and Apple",
"The work is to write some papers in science"]

tfidf = TfidfVectorizer()
re = tfidf.fit_transform(corpus)
print(re)
